<a href="https://colab.research.google.com/github/nicop1709/projet8-cityscapes/blob/main/P8_cityscapes_segmentation_pipeline_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Projet 8 — Pipeline complet Cityscapes (8 classes) : Entraînement, Évaluation et Prédiction

Version sans `tensorflow-addons` (compat. Colab standard).

- Chargement Cityscapes (chemins Drive)
- Remap → 8 classes
- `tf.data` + augmentations
- Modèles : **U-Net** et **U-Net+MobileNetV2**
- mIoU (callback maison), export **SavedModel**
- Artefacts pour API (config, palette, LUT, metrics)
- Prédiction + visualisation


## 0. Environnement & dépendances

In [2]:
!pip install --quiet mlflow dagshub albumentations


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.9/753.9 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/2

In [3]:

# Pas de tensorflow-addons requis
import os, sys, json, math, time, itertools, random
from pathlib import Path
from pprint import pprint

import numpy as np
import tensorflow as tf
import pandas as pd
print("TensorFlow:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))
import albumentations as A
import mlflow
import dagshub


TensorFlow: 2.19.0
GPU available: []


## 1. Chemins & configuration

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [81]:

PATH_PROJECT = '/content/drive/MyDrive/AI Engineer/Projets/Projet 8 - Segmentation images voitures autonomes'

FOLDER_TRAINING_IMAGES = PATH_PROJECT + "/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit/train"
FOLDER_TRAINING_MASK   = PATH_PROJECT + "/P8_Cityscapes_gtFine_trainvaltest/gtFine/train"
FOLDER_VALIDATION_IMAGES = PATH_PROJECT + "/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit/val"
FOLDER_VALIDATION_MASK   = PATH_PROJECT + "/P8_Cityscapes_gtFine_trainvaltest/gtFine/val"
FOLDER_TEST_IMAGES = PATH_PROJECT + "/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit/test"
FOLDER_TEST_MASK   = PATH_PROJECT + "/P8_Cityscapes_gtFine_trainvaltest/gtFine/test"

IMG_HEIGHT, IMG_WIDTH = 512, 1024
N_CLASSES = 8
BATCH_SIZE = 4
EPOCHS = 25
SEED = 42

OUT_DIR = Path(PATH_PROJECT) / "artifacts_training"
OUT_DIR.mkdir(parents=True, exist_ok=True)

tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

print("OUT_DIR:", OUT_DIR)


OUT_DIR: /content/drive/MyDrive/AI Engineer/Projets/Projet 8 - Segmentation images voitures autonomes/artifacts_training


In [45]:
DAGSHUB_USERNAME = 'npai.consulting'
DAGSHUB_REPO = 'p08'
DAGSHUB_TOKEN = '0b9784c9193bb3daff1a6ae6d8008af830109b0b'

if DAGSHUB_USERNAME and DAGSHUB_REPO:
    print(f"Initialisation Dagshub pour {DAGSHUB_USERNAME}/{DAGSHUB_REPO}")
    dagshub.init(repo_name=DAGSHUB_REPO, repo_owner=DAGSHUB_USERNAME, mlflow=True)
    tracking_uri = os.environ.get('MLFLOW_TRACKING_URI', f"https://dagshub.com/{DAGSHUB_USERNAME}/{DAGSHUB_REPO}.mlflow")
else:
    tracking_uri = os.environ.get('MLFLOW_TRACKING_URI', f"file://{OUT_DIR / 'mlruns'}")

mlflow.set_tracking_uri(tracking_uri)
EXPERIMENT_NAME = os.environ.get('MLFLOW_EXPERIMENT_NAME', 'cityscapes-segmentation')
mlflow.set_experiment(EXPERIMENT_NAME)
print('MLflow tracking URI:', mlflow.get_tracking_uri())


Initialisation Dagshub pour npai.consulting/p08


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=dd358dd4-0a0c-43cd-9fc7-59d17b5b4475&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=0c39e8e74572df3f2f7b6083f745e159a7182cf9f8c69090b105dfd5f7946210




Accessing as npai.consulting

Initialized MLflow to track repo "npai.consulting/p08"

Repository npai.consulting/p08 initialized!

2025/11/20 07:54:19 INFO mlflow.tracking.fluent: Experiment with name 'cityscapes-segmentation' does not exist. Creating a new experiment.


MLflow tracking URI: https://dagshub.com/npai.consulting/p08.mlflow


In [46]:
for path in [FOLDER_TRAINING_IMAGES,
             FOLDER_TRAINING_MASK,
             FOLDER_VALIDATION_IMAGES,
             FOLDER_VALIDATION_MASK,
             FOLDER_TEST_IMAGES,
             FOLDER_TEST_MASK
             ]:
  print(os.listdir(path))

['zurich', 'strasbourg', 'weimar', 'tubingen', 'jena', 'aachen', 'bochum', 'dusseldorf', 'darmstadt', 'cologne', 'hamburg', 'krefeld', 'monchengladbach', 'hanover', 'stuttgart', 'ulm', 'erfurt', 'bremen']
['weimar', 'tubingen', 'zurich', 'aachen', 'strasbourg', 'bochum', 'jena', 'dusseldorf', 'ulm', 'darmstadt', 'hamburg', 'cologne', 'monchengladbach', 'stuttgart', 'erfurt', 'hanover', 'krefeld', 'bremen']
['munster', 'frankfurt', 'lindau']
['lindau', 'munster', 'frankfurt']
['bielefeld', 'bonn', 'leverkusen', 'berlin', 'mainz', 'munich']
['leverkusen', 'bielefeld', 'bonn', 'berlin', 'mainz', 'munich']


In [8]:
image_counts = []

# Define the image folders to process
image_folders = {
    "train": FOLDER_TRAINING_IMAGES,
    "val": FOLDER_VALIDATION_IMAGES,
    "test": FOLDER_TEST_IMAGES
}

# Iterate through each dataset split (train, val, test)
for split_name, folder_path in image_folders.items():
    print(f"[INFO]: Processing {split_name} images in {folder_path}")
    # Iterate through each city subfolder in the current split
    for city_folder in os.listdir(folder_path):
        city_folder_full_path = os.path.join(folder_path, city_folder)
        if os.path.isdir(city_folder_full_path):
            # Count the number of image files (assuming .png) in the city folder
            num_images = len([f for f in os.listdir(city_folder_full_path) if f.endswith('.png')])
            image_counts.append({
                "split": split_name,
                "city": city_folder,
                "image_count": num_images
            })

# Create a DataFrame from the collected data
images_per_city_df = pd.DataFrame(image_counts)

print("\nDataFrame of image counts per city and split:")
images_per_city_df

[INFO]: Processing train images in /content/drive/MyDrive/AI Engineer/Projets/Projet 8 - Segmentation images voitures autonomes/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit/train
[INFO]: Processing val images in /content/drive/MyDrive/AI Engineer/Projets/Projet 8 - Segmentation images voitures autonomes/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit/val
[INFO]: Processing test images in /content/drive/MyDrive/AI Engineer/Projets/Projet 8 - Segmentation images voitures autonomes/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit/test

DataFrame of image counts per city and split:


,split,city,image_count
0,train,zurich,122
1,train,strasbourg,365
2,train,weimar,142
3,train,tubingen,144
4,train,jena,119
5,train,aachen,174
6,train,bochum,96
7,train,dusseldorf,221
8,train,darmstadt,85
9,train,cologne,154


In [9]:
# Nb of images by split
images_per_city_df.groupby(["split"])['image_count'].sum()


,image_count
split,
test,1525
train,2975
val,500


In [10]:
# Define a dataframe with Pandas
dataset_df = pd.DataFrame()

if os.path.exists(PATH_PROJECT+'/train.csv'):
  dataset_df = pd.read_csv(PATH_PROJECT+"/train.csv",
                         converters={"labels": json.loads, "labels_main": json.loads},index_col="image_id")
  dataset_df.index.name = 'index'
else:
  # Loop over city folder
  for city_folder in os.listdir(FOLDER_TRAINING_MASK):
      print("[INFO]: Processing city folder ", city_folder)

      city_folder_full = os.path.join(FOLDER_TRAINING_MASK, city_folder)

      # Loop in each city folder
      for i, file in enumerate(os.listdir(city_folder_full)):

          # Parse only json file
          if file.endswith("json"):
              print(f"[INFO]: Processing file {i+1} / {len(os.listdir(city_folder_full))}: {file}")
              filename = os.path.join(city_folder_full, file)

              # In each json file, pop labels
              with open(filename) as json_file:
                  data = json.load(json_file)
                  labels = list()
                  labels_main = list()

                  for elt in data["objects"]:
                      for k, v in elt.items():
                          if k == "label":
                              labels.append(v)
                              try:
                                  labels_main.append(name2label[v][3])
                              except:
                                  pass

                  data["labels"] = labels
                  data["labels_count"] = len(labels)

                  data["labels_main"] = labels_main
                  data["labels_main_count"] = len(labels_main)

                  data["city"] = city_folder
                  del data["objects"]

              basename = "_".join(file.split(".")[0].split("_")[:3])

              df_tmp = pd.DataFrame.from_dict(data, orient='index', columns=[basename]).T
              dataset_df = pd.concat([dataset_df, df_tmp])
              (dataset_df
    .assign(labels=lambda d: d["labels"].apply(json.dumps),
            labels_main=lambda d: d["labels_main"].apply(json.dumps))
    .to_csv(PATH_PROJECT+"/train.csv", index=True,index_label="image_id"))

In [11]:
dataset_df

,imgHeight,imgWidth,labels,labels_count,labels_main,labels_main_count,city
index,,,,,,,
weimar_000012_000019,1024,2048,"[road, sidewalk, sidewalk, vegetation, sky, fe...",69,"[flat, flat, flat, nature, sky, construction, ...",69,weimar
weimar_000052_000019,1024,2048,"[sky, road, sidewalk, sidewalk, building, buil...",43,"[sky, flat, flat, flat, construction, construc...",43,weimar
weimar_000044_000019,1024,2048,"[sky, road, building, vegetation, static, side...",43,"[sky, flat, construction, nature, void, flat, ...",42,weimar
weimar_000009_000019,1024,2048,"[road, sky, sidewalk, sidewalk, building, vege...",47,"[flat, sky, flat, flat, construction, nature, ...",46,weimar
weimar_000040_000019,1024,2048,"[sky, road, vegetation, building, pole, static...",48,"[sky, flat, nature, construction, object, void...",47,weimar
...,...,...,...,...,...,...,...
bremen_000080_000019,1024,2048,"[sky, road, static, static, static, static, st...",85,"[sky, flat, void, void, void, void, void, natu...",84,bremen
bremen_000076_000019,1024,2048,"[sky, sidewalk, road, building, sidewalk, buil...",76,"[sky, flat, flat, construction, flat, construc...",74,bremen
bremen_000062_000019,1024,2048,"[static, road, ground, pole, parking, car, per...",52,"[void, flat, void, object, flat, vehicle, huma...",51,bremen


## 2. Remapping Cityscapes → 8 classes

In [12]:
import numpy as np
import cv2
# ------------------------------------------------------------------
# Groupes d'IDs Cityscapes (id dans les *_gtFine_labelIds.png)
# regroupés par grande catégorie (8 classes)
# ------------------------------------------------------------------
void = [
    0,  # unlabeled
    1,  # ego vehicle
    2,  # rectification border
    3,  # out of roi
    4,  # static
    5,  # dynamic
    6,  # ground
]

flat = [
    7,  # road
    8,  # sidewalk
    9,  # parking
    10, # rail track
]

construction = [
    11, # building
    12, # wall
    13, # fence
    14, # guard rail
    15, # bridge
    16, # tunnel
]

object = [
    17, # pole
    18, # polegroup
    19, # traffic light
    20, # traffic sign
]

nature = [
    21, # vegetation
    22, # terrain
]

sky = [
    23, # sky
]

human = [
    24, # person
    25, # rider
]

vehicle = [
    26, # car
    27, # truck
    28, # bus
    29, # caravan
    30, # trailer
    31, # train
    32, # motorcycle
    33, # bicycle
    # id = -1 (license plate) n'est pas inclus
]

# ------------------------------------------------------------------
# REMAPPING vers 8 classes :
# 0: void, 1: flat, 2: construction, 3: object,
# 4: nature, 5: sky, 6: human, 7: vehicle
# ------------------------------------------------------------------
REMAPPING = {
    0: void,
    1: flat,
    2: construction,
    3: object,
    4: nature,
    5: sky,
    6: human,
    7: vehicle,
}

# LUT 256 -> 8 classes
lut = np.zeros(256, dtype=np.uint8)
for new_id, src_ids in REMAPPING.items():
    for sid in src_ids:
        lut[sid] = new_id

PALETTE = np.array([
    [  0,   0,   0],   # 0 void
    [128,  64, 128],   # 1 flat        (road)
    [ 70,  70,  70],   # 2 construction (building)
    [220, 220,   0],   # 3 object      (traffic sign style)
    [107, 142,  35],   # 4 nature      (vegetation)
    [ 70, 130, 180],   # 5 sky
    [220,  20,  60],   # 6 human       (person)
    [  0,   0, 142],   # 7 vehicle     (car)
], dtype=np.uint8)

# Noms des 8 classes dans l’ordre de ton mapping
LABELS = [
    "void",
    "flat",
    "construction",
    "object",
    "nature",
    "sky",
    "human",
    "vehicle",
]

# Dossier de sortie pour les fichiers de mapping
OUT_DIR = Path("mappings")  # ou Path(PATH_PROJECT) / "mappings" si tu préfères
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Sauvegarde des métadonnées de classes
with open(OUT_DIR / "labels.json", "w") as f:
    json.dump({"labels": LABELS}, f, indent=2, ensure_ascii=False)

# Sauvegarde de la LUT (256 -> 8) et de la palette (8 x 3)
np.save(OUT_DIR / "lut_256_to_8.npy", lut)
np.save(OUT_DIR / "palette_8x3.npy", PALETTE)

print("Mapping & palette sauvegardés dans", OUT_DIR)

Mapping & palette sauvegardés dans mappings


## 3. Indexation des fichiers & correspondance image/mask

In [13]:
def gather_pairs(images_root, masks_root):
    """
    Associe:
      .../leftImg8bit/<split>/<city>/<frame>_leftImg8bit.png
    avec:
      .../gtFine/<split>/<city>/<frame>_gtFine_labelIds.png
    """
    images_root = Path(images_root)
    masks_root  = Path(masks_root)
    img_paths, mask_paths = [], []

    # parcourt toutes les images *_leftImg8bit.png (toutes villes)
    for img_path in images_root.rglob("*_leftImg8bit.png"):
        if not img_path.is_file():
            continue
        city = img_path.parent.name  # dossier ville
        base = img_path.name[:-len("_leftImg8bit.png")]  # <frame>
        mask_path = masks_root / city / f"{base}_gtFine_labelIds.png"
        if mask_path.exists():
            img_paths.append(str(img_path))
            mask_paths.append(str(mask_path))
        else:
            # fallback (au cas où)
            cand = list((masks_root / city).glob(f"{base}_gtFine_labelIds.png"))
            if cand:
                img_paths.append(str(img_path))
                mask_paths.append(str(cand[0]))

    return img_paths, mask_paths


train_imgs, train_msks = gather_pairs(FOLDER_TRAINING_IMAGES, FOLDER_TRAINING_MASK)
val_imgs,   val_msks   = gather_pairs(FOLDER_VALIDATION_IMAGES, FOLDER_VALIDATION_MASK)
test_imgs,  test_msks  = gather_pairs(FOLDER_TEST_IMAGES,      FOLDER_TEST_MASK)

print("Train pairs:", len(train_imgs))
print("Val   pairs:", len(val_imgs))
print("Test  pairs:", len(test_imgs))

with open(OUT_DIR / "index_train.json", "w") as f:
    json.dump({"images": train_imgs, "masks": train_msks}, f, indent=2)
with open(OUT_DIR / "index_val.json", "w") as f:
    json.dump({"images": val_imgs, "masks": val_msks}, f, indent=2)
with open(OUT_DIR / "index_test.json", "w") as f:
    json.dump({"images": test_imgs, "masks": test_msks}, f, indent=2)


Train pairs: 2975
Val   pairs: 500
Test  pairs: 1525


## 4. Pipeline tf.data (+ augmentations)

In [14]:
AUTOTUNE = tf.data.AUTOTUNE
lut_tf = tf.constant(np.load(str(OUT_DIR / "lut_256_to_8.npy")), dtype=tf.uint8)

albumentations_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, border_mode=cv2.BORDER_REFLECT_101, p=0.3),
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),
    A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=10, p=0.2),
    A.GaussianBlur(blur_limit=(3, 5), p=0.15),
    A.Resize(height=IMG_HEIGHT, width=IMG_WIDTH, interpolation=cv2.INTER_LINEAR), # Ensure final size is correct
])

def _load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

def _load_mask(mask_path):
    m = tf.io.read_file(mask_path)
    m = tf.image.decode_png(m, channels=1)
    m = tf.squeeze(m, axis=-1)
    m = tf.gather(lut_tf, tf.cast(m, tf.int32))
    m = tf.expand_dims(m, axis=-1)
    return m

def _resize(img, msk):
    img = tf.image.resize(img, (IMG_HEIGHT, IMG_WIDTH), method="bilinear")
    msk = tf.image.resize(msk, (IMG_HEIGHT, IMG_WIDTH), method="nearest")
    return img, msk

def _apply_albumentations(img, msk):
    def _numpy_aug(image, mask):
        image_np = image.numpy()
        mask_np = np.squeeze(mask.numpy()).astype(np.int32)
        augmented = albumentations_transform(image=image_np, mask=mask_np)
        aug_img = augmented["image"].astype(np.float32)
        aug_mask = np.expand_dims(augmented["mask"].astype(np.int32), axis=-1)
        return aug_img, aug_mask
    aug_img, aug_mask = tf.py_function(_numpy_aug, [img, msk], [tf.float32, tf.int32])
    aug_img.set_shape((IMG_HEIGHT, IMG_WIDTH, 3))
    aug_mask.set_shape((IMG_HEIGHT, IMG_WIDTH, 1))
    aug_img = tf.clip_by_value(aug_img, 0.0, 1.0)
    return aug_img, aug_mask

def _prep_train(img_path, msk_path):
    img = _load_image(img_path)
    msk = _load_mask(msk_path)
    img, msk = _resize(img, msk)
    img, msk = _apply_albumentations(img, msk)
    return img, tf.cast(msk, tf.int32)

def _prep_eval(img_path, msk_path):
    img = _load_image(img_path)
    msk = _load_mask(msk_path)
    img, msk = _resize(img, msk)
    return img, tf.cast(msk, tf.int32)

def make_ds(imgs, msks, training=True, batch_size=BATCH_SIZE):
    imgs = [str(p) for p in imgs]
    msks = [str(p) for p in msks]

    imgs = tf.constant(imgs, dtype=tf.string)
    msks = tf.constant(msks, dtype=tf.string)

    ds = tf.data.Dataset.from_tensor_slices((imgs, msks))
    if training:
        buf = int(min(1000, imgs.shape[0]))
        if buf > 0:
            ds = ds.shuffle(buf, seed=SEED, reshuffle_each_iteration=True)
    ds = ds.map(_prep_train if training else _prep_eval, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

train_ds = make_ds(train_imgs, train_msks, training=True)
val_ds   = make_ds(val_imgs,   val_msks,   training=False)
test_ds  = make_ds(test_imgs,  test_msks,  training=False)

for batch in train_ds.take(1):
    print("Batch images:", batch[0].shape, "Batch masks:", batch[1].shape)


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Batch images: (4, 512, 1024, 3) Batch masks: (4, 512, 1024, 1)


## 5. Pertes & métriques (mIoU callback)

In [15]:
from tensorflow.keras import backend as K

CE_LOSS = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

class MeanIoUCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_dataset, name="val_mIoU"):
        super().__init__()
        self.val_dataset = val_dataset
        self.name = name

    def on_epoch_end(self, epoch, logs=None):
        miou = tf.keras.metrics.MeanIoU(num_classes=N_CLASSES)
        for x, y in self.val_dataset:
            pred = self.model.predict(x, verbose=0)
            pred_labels = tf.argmax(pred, axis=-1)
            pred_labels = tf.expand_dims(pred_labels, axis=-1)
            miou.update_state(y, pred_labels)
        logs = logs or {}
        logs[self.name] = miou.result().numpy()
        # The following line was causing the ValueError because Keras already handles this.
        # self.model.history.history.setdefault(self.name, []).append(logs[self.name])
        print(f" — {self.name}: {logs[self.name]:.4f}")

## 6. Modèles : U-Net et U-Net-MobileNetV2

In [34]:
from tensorflow.keras import layers, models

def conv_block(x, filters, k=3):
    x = layers.Conv2D(filters, k, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, k, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def encoder_block(x, filters):
    c = conv_block(x, filters)
    p = layers.MaxPooling2D((2,2))(c)
    return c, p

def decoder_block(x, skip, filters):
    x = layers.Conv2DTranspose(filters, (2,2), strides=(2,2), padding="same")(x)
    x = layers.Concatenate()([x, skip])
    x = conv_block(x, filters)
    return x

def build_unet(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=N_CLASSES, base_filters=64):
    inputs = layers.Input(shape=input_shape)
    c1, p1 = encoder_block(inputs, base_filters)
    c2, p2 = encoder_block(p1, base_filters*2)
    c3, p3 = encoder_block(p2, base_filters*4)
    c4, p4 = encoder_block(p3, base_filters*8)

    bn = conv_block(p4, base_filters*16)

    d1 = decoder_block(bn, c4, base_filters*8)
    d2 = decoder_block(d1, c3, base_filters*4)
    d3 = decoder_block(d2, c2, base_filters*2)
    d4 = decoder_block(d3, c1, base_filters)

    outputs = layers.Conv2D(num_classes, 1, activation="softmax")(d4)
    return models.Model(inputs, outputs, name="UNet")

def build_unet_mobilenetv2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=N_CLASSES):
    base = tf.keras.applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights="imagenet"
    )

    # Couches de skip-connection
    skips = [
        base.get_layer("block_1_expand_relu").output,   # ~ 1/2
        base.get_layer("block_3_expand_relu").output,   # ~ 1/4
        base.get_layer("block_6_expand_relu").output,   # ~ 1/8
        base.get_layer("block_13_expand_relu").output,  # ~ 1/16
    ]
    encoder_output = base.get_layer("block_16_project").output  # ~ 1/32

    x = encoder_output

    # Decoder : 1/32 → 1/16 → 1/8 → 1/4 → 1/2
    filters = [256, 128, 64, 32]
    for i, f in enumerate(filters):
        x = layers.Conv2DTranspose(f, 3, strides=2, padding="same")(x)
        x = layers.Concatenate()([x, skips[-(i+1)]])
        x = layers.Conv2D(f, 3, padding="same", activation="relu")(x)
        x = layers.Conv2D(f, 3, padding="same", activation="relu")(x)

    # Dernier upsampling 1/2 → 1 (512 x 1024)
    x = layers.Conv2DTranspose(16, 3, strides=2, padding="same")(x)
    x = layers.Conv2D(16, 3, padding="same", activation="relu")(x)

    outputs = layers.Conv2D(num_classes, 1, activation="softmax")(x)
    model = models.Model(inputs=base.input, outputs=outputs, name="UNet_MobileNetV2")
    return model

unet = build_unet()
unet_mb = build_unet_mobilenetv2()
unet.summary()
unet_mb.summary()

config = {
    "img_height": IMG_HEIGHT,
    "img_width": IMG_WIDTH,
    "n_classes": N_CLASSES,
    "labels": LABELS,
    "preprocess": "rescale_0_1",
    "color_palette": "palette_8x3.npy"
}
with open(OUT_DIR / "inference_config.json", "w") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)
print("Modèles définis et config d'inférence sauvegardée.")

/tmp/ipython-input-2027914468.py:41: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = tf.keras.applications.MobileNetV2(


Model: "UNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 512, 1024, │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_46 (Conv2D)  │ (None, 512, 1024, │      1,792 │ input_layer_8[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512, 1024, │        256 │ conv2d_46[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_18 (ReLU)     │ (None, 512, 1024, │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_47 (Conv2D)  │ (None, 512, 1024, │     36,928 │ re_lu_18[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512, 1024, │        256 │ conv2d_47[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_19 (ReLU)     │ (None, 512, 1024, │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_15    │ (None, 256, 512,  │          0 │ re_lu_19[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_48 (Conv2D)  │ (None, 256, 512,  │     73,856 │ max_pooling2d_15… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 512,  │        512 │ conv2d_48[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_20 (ReLU)     │ (None, 256, 512,  │          0 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_49 (Conv2D)  │ (None, 256, 512,  │    147,584 │ re_lu_20[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 512,  │        512 │ conv2d_49[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_21 (ReLU)     │ (None, 256, 512,  │          0 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_16    │ (None, 128, 256,  │          0 │ re_lu_21[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_50 (Conv2D)  │ (None, 128, 256,  │    295,168 │ max_pooling2d_16… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 256,  │      1,024 │ conv2d_50[0][0] 

 Total params: 31,055,752 (118.47 MB)

 Trainable params: 31,043,976 (118.42 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 512, 1024, │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 256, 512,  │        864 │ input_layer_9[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 256, 512,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 256, 512,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 256, 512,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 256, 512,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 256, 512,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 256, 512,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 256, 512,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 256, 512,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 256, 512,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 256, 512,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 257, 513,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 128, 256,  │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 128, 256,  │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 128, 256,  │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 128, 256,  │      2,304 │ block_1_depthwis

 Total params: 6,200,456 (23.65 MB)

 Trainable params: 6,169,544 (23.53 MB)

 Non-trainable params: 30,912 (120.75 KB)

Modèles définis et config d'inférence sauvegardée.


## 7. Entraînement & suivi mIoU

In [76]:
def compile_model(model, lr=1e-3):
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt,
                  loss=CE_LOSS,
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="acc")])
    return model

callbacks_common = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2),
    tf.keras.callbacks.CSVLogger(str(OUT_DIR / "training_log.csv"), append=True)
]

def log_history_to_mlflow(history, prefix=""):
    for metric_name, values in history.history.items():
        log_name = f"{prefix}_{metric_name}" if prefix else metric_name
        for step, value in enumerate(values):
            mlflow.log_metric(log_name, float(value), step=step)

def train_model_with_mlflow(model, model_name, callbacks_extra=None):
    callbacks_extra = callbacks_extra or []
    callbacks = callbacks_common + callbacks_extra
    dataset_csv = Path(PATH_PROJECT) / "train.csv"
    run_name = f"{model_name}-training"
    tags = {"model": model_name, "stage": "training"}

    with mlflow.start_run(run_name=run_name, tags=tags) as run:
        # Log des hyperparams et infos de dataset
        mlflow.log_params({
            "img_height": IMG_HEIGHT,
            "img_width": IMG_WIDTH,
            "n_classes": N_CLASSES,
            "batch_size": BATCH_SIZE,
            "epochs": EPOCHS,
            "train_size": len(train_imgs),
            "val_size": len(val_imgs),
            "augmentations": "albumentations_compose"
        })

        if dataset_csv.exists():
            mlflow.log_artifact(str(dataset_csv), artifact_path="dataset")

        config_path = OUT_DIR / "inference_config.json"
        if config_path.exists():
            mlflow.log_artifact(str(config_path), artifact_path="config")

        # Entraînement
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=EPOCHS,
            callbacks=callbacks,
            verbose=1
        )

        # Sauvegarde de l'historique en local
        history_path = OUT_DIR / f"history_{model_name}.csv"
        pd.DataFrame(history.history).to_csv(history_path, index=False)

        # Log des courbes dans MLflow
        log_history_to_mlflow(history)
        mlflow.log_artifact(str(history_path), artifact_path="training_curves")

        # Log du meilleur checkpoint (callback ModelCheckpoint)
        best_ckpt = OUT_DIR / f"best_{model_name}.keras"
        if best_ckpt.exists():
            mlflow.log_artifact(str(best_ckpt), artifact_path="checkpoints")

        # Sauvegarde du modèle final au format natif Keras + log en tant qu'artefact
        with tempfile.TemporaryDirectory() as tmpdir:
            local_model_path = os.path.join(tmpdir, f"{model_name}.keras")
            model.save(local_model_path)  # format .keras (Keras 3)
            # On log le fichier du modèle comme artefact (compat DagsHub)
            mlflow.log_artifact(local_model_path, artifact_path="model")

        return run.info.run_id, history, history_path, model_name, model

In [18]:
MLFLOW_TRAINING_RUN_IDS = {}

In [77]:
def build_simple_test_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=N_CLASSES):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax', padding='same')(x) # Output layer with N_CLASSES channels
    # Upsample to original resolution
    x = tf.keras.layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(x) # Upsample to original IMG_HEIGHT/IMG_WIDTH

    model = tf.keras.models.Model(inputs=inputs, outputs=x, name="SimpleTestModel")
    return model

simple_model = build_simple_test_model()
simple_model.summary()

print("Train Simple Test Model...")
simple_model = compile_model(simple_model, lr=1e-3)
miou_cb_simple = MeanIoUCallback(val_ds, name="val_mIoU")
ckpt_simple = tf.keras.callbacks.ModelCheckpoint(
    filepath=str(OUT_DIR / "best_simple_test_model.keras"),
    monitor="val_loss",
    save_best_only=True
)

# Temporarily reduce epochs for a quick test run
original_epochs = EPOCHS
# You might want to adjust this to 1 or 2 for a very quick test
# For this demonstration, I will use a small number of epochs (e.g., 2)
TEST_EPOCHS = 2

# Store the original EPOCHS value and restore it later if needed
globals()['EPOCHS'] = TEST_EPOCHS

# Ensure no active run exists from previous interrupted executions
if mlflow.active_run():
    print(f"Ending active run: {mlflow.active_run().info.run_id}")
    mlflow.end_run()

# Unpack the 5 returned values correctly
run_id_simple, history_simple, _, _, _ = train_model_with_mlflow(
    simple_model, "simple_test_model", callbacks_extra=[miou_cb_simple, ckpt_simple]
)
MLFLOW_TRAINING_RUN_IDS["simple_test_model"] = run_id_simple

# Restore original EPOCHS value
globals()['EPOCHS'] = original_epochs

print("Simple Test Model training complete and logged with MLflow.")

Model: "SimpleTestModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, 512, 1024, 3)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_81 (Conv2D)              │ (None, 512, 1024, 32)  │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 256, 512, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_82 (Conv2D)              │ (None, 256, 512, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 128, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_83 (Conv2D)              │ (None, 128, 256, 8)    │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_8 (UpSampling2D)  │ (None, 512, 1024, 8)   │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,912 (77.78 KB)

 Trainable params: 19,912 (77.78 KB)

 Non-trainable params: 0 (0.00 B)

Train Simple Test Model...
Ending active run: dfbcb3881c5d43ae808c97bdc57dc8db
🏃 View run unleashed-hare-33 at: https://dagshub.com/npai.consulting/p08.mlflow/#/experiments/0/runs/dfbcb3881c5d43ae808c97bdc57dc8db
🧪 View experiment at: https://dagshub.com/npai.consulting/p08.mlflow/#/experiments/0
Epoch 1/2
744/744 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - acc: 0.4436 - loss: 1.5265 — val_mIoU: 0.2814
744/744 ━━━━━━━━━━━━━━━━━━━━ 940s 1s/step - acc: 0.4437 - loss: 1.5262 - val_acc: 0.6170 - val_loss: 1.1199 - learning_rate: 0.0010 - val_mIoU: 0.2814
Epoch 2/2
744/744 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - acc: 0.5943 - loss: 1.1671 — val_mIoU: 0.2946
744/744 ━━━━━━━━━━━━━━━━━━━━ 949s 1s/step - acc: 0.5943 - loss: 1.1671 - val_acc: 0.6276 - val_loss: 1.0768 - learning_rate: 0.0010 - val_mIoU: 0.2946
🏃 View run simple_test_model-training at: https://dagshub.com/npai.consulting/p08.mlflow/#/experiments/0/runs/4f92d5025215431db07822a267c5ca77
🧪 View experiment at: https://dagshub.com/npai.consulting/p08

In [ ]:
print("Train UNet...")
unet = compile_model(unet, lr=1e-3)
miou_cb_unet = MeanIoUCallback(val_ds, name="val_mIoU")
ckpt_unet = tf.keras.callbacks.ModelCheckpoint(
    filepath=str(OUT_DIR / "best_unet.keras"),
    monitor="val_loss",
    save_best_only=True
)
run_id_unet, history_unet = train_model_with_mlflow(unet, "unet", callbacks_extra=[miou_cb_unet, ckpt_unet])
MLFLOW_TRAINING_RUN_IDS["unet"] = run_id_unet

print("Historiques sauvegardés et suivis avec MLflow.")

In [ ]:
print("Train UNet-MobileNetV2...")
unet_mb = compile_model(unet_mb, lr=1e-3)
miou_cb_unetmb = MeanIoUCallback(val_ds, name="val_mIoU")
ckpt_unetmb = tf.keras.callbacks.ModelCheckpoint(
    filepath=str(OUT_DIR / "best_unet_mobilenetv2.keras"),
    monitor="val_loss",
    save_best_only=True
)
run_id_unetmb, history_unet_mb = train_model_with_mlflow(unet_mb, "unet_mobilenetv2", callbacks_extra=[miou_cb_unetmb, ckpt_unetmb])
MLFLOW_TRAINING_RUN_IDS["unet_mobilenetv2"] = run_id_unetmb

print("Historiques sauvegardés et suivis avec MLflow.")

## 8. Sélection du meilleur modèle & export

In [ ]:

# import pandas as pd

# def best_miou(csv_path, key="val_mIoU"):
#     p = Path(csv_path)
#     if not p.exists():
#         return -1.0
#     df = pd.read_csv(p)
#     return float(df[key].max()) if key in df.columns else -1.0

# miou_unet = best_miou(OUT_DIR / "history_unet.csv")
# miou_unet_mb = best_miou(OUT_DIR / "history_unet_mobilenetv2.csv")

# print("Best val mIoU — UNet:", miou_unet)
# print("Best val mIoU — UNet-MobileNetV2:", miou_unet_mb)

# if miou_unet_mb > miou_unet:
#     best_name = "unet_mobilenetv2"
#     best_ckpt = OUT_DIR / "best_unet_mobilenetv2.keras"
#     model = tf.keras.models.load_model(best_ckpt, compile=False)
# else:
#     best_name = "unet"
#     best_ckpt = OUT_DIR / "best_unet.keras"
#     model = tf.keras.models.load_model(best_ckpt, compile=False)

# BEST_EXPORT_DIR = OUT_DIR / f"savedmodel_{best_name}"
# tf.keras.models.save_model(model, BEST_EXPORT_DIR, include_optimizer=False, save_format="tf")
# print("Export:", BEST_EXPORT_DIR)

# with open(OUT_DIR / "best_model.json", "w") as f:
#     json.dump({
#         "best_model": best_name,
#         "checkpoint": str(best_ckpt),
#         "export_dir": str(BEST_EXPORT_DIR),
#         "val_mIoU_unet": miou_unet,
#         "val_mIoU_unet_mobilenetv2": miou_unet_mb
#     }, f, indent=2)


In [93]:
from pathlib import Path
import mlflow
import tensorflow as tf

def load_model_from_mlflow(run_id: str):
    """
    Télécharge les artefacts (artifact_path='model') d'un run MLflow
    et charge le premier fichier .keras trouvé comme modèle Keras.
    """
    local_dir = mlflow.artifacts.download_artifacts(
        run_id=run_id,
        artifact_path="model"
    )
    local_dir = Path(local_dir)

    # Lister tous les fichiers .keras dans ce dossier (et sous-dossiers au cas où)
    keras_files = list(local_dir.rglob("*.keras"))

    if not keras_files:
        # Debug : montrer ce qu'il y a dans le dossier
        all_files = [str(p) for p in local_dir.rglob("*")]
        raise FileNotFoundError(
            "Aucun fichier .keras trouvé dans les artefacts MLflow.\n"
            f"Dossier téléchargé : {local_dir}\n"
            f"Contenu trouvé :\n" + "\n".join(all_files)
        )

    model_path = keras_files[0]
    print(f"Fichier modèle trouvé : {model_path}")
    model = tf.keras.models.load_model(model_path, compile=False)
    return model, model_path


In [97]:
import json
import shutil

best_name = "simple_model"
print("Chargement du modèle depuis MLflow pour les tests...")

model_from_mlflow, model_path = load_model_from_mlflow(run_id_simple)
print("Modèle chargé depuis :", model_path)

BEST_EXPORT_DIR = OUT_DIR / f"savedmodel_{best_name}"
if BEST_EXPORT_DIR.exists():
    shutil.rmtree(BEST_EXPORT_DIR)

model_from_mlflow.export(str(BEST_EXPORT_DIR))
print("Export SavedModel dans :", BEST_EXPORT_DIR)

best_info = {
    "best_model": best_name,
    "checkpoint": str(model_path),
    "export_dir": str(BEST_EXPORT_DIR),
    "source_run_id": run_id_simple,
}

with open(OUT_DIR / "best_model.json", "w") as f:
    json.dump(best_info, f, indent=2)

print("Résumé sauvegardé dans best_model.json")


Chargement du modèle depuis MLflow pour les tests...


Fichier modèle trouvé : /tmp/tmph0rrdgmi/model/simple_test_model.keras
Modèle chargé depuis : /tmp/tmph0rrdgmi/model/simple_test_model.keras
Saved artifact at '/content/drive/MyDrive/AI Engineer/Projets/Projet 8 - Segmentation images voitures autonomes/artifacts_training/savedmodel_simple_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 512, 1024, 3), dtype=tf.float32, name='input_layer_12')]
Output Type:
  TensorSpec(shape=(None, 512, 1024, 8), dtype=tf.float32, name=None)
Captures:
  136787727310800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136787727314832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136787727313872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136787727315024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136787727313488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136787727313104: TensorSpec(shape=(), dtype=tf.resource, name=None)
Export SavedMode

## 9. Évaluation test

In [101]:
with open(OUT_DIR / "best_model.json") as f:
    meta = json.load(f)

best_name = meta["best_model"]

# Chemin vers le .keras (et pas export_dir !)
best_ckpt = Path(meta["checkpoint"])

print("Best model:", best_name)
print("Checkpoint (.keras) :", best_ckpt)

best_model = tf.keras.models.load_model(best_ckpt, compile=False)
print("Modèle chargé pour les tests.")


miou_test = tf.keras.metrics.MeanIoU(num_classes=N_CLASSES)
acc_test = tf.keras.metrics.SparseCategoricalAccuracy()

for x, y in test_ds:
    pred = best_model.predict(x, verbose=0)
    acc_test.update_state(y, pred)
    pred_labels = tf.argmax(pred, axis=-1)
    pred_labels = tf.expand_dims(pred_labels, axis=-1)
    miou_test.update_state(y, pred_labels)

test_miou = float(miou_test.result().numpy())
test_acc = float(acc_test.result().numpy())

metrics_path = OUT_DIR / "metrics_test.json"
metrics_payload = {
    "test_mIoU": test_miou,
    "test_acc": test_acc,
    "test_samples": len(test_imgs)
}
with open(metrics_path, "w") as f:
    json.dump(metrics_payload, f, indent=2)
print("Métriques test sauvegardées.")

parent_run = MLFLOW_TRAINING_RUN_IDS.get(best_name)
eval_tags = {"stage": "evaluation", "best_model": best_name}
if parent_run:
    eval_tags["parent_run_id"] = parent_run
with mlflow.start_run(run_name=f"{best_name}-evaluation", tags=eval_tags) as eval_run:
    mlflow.log_metrics({"test_mIoU": test_miou, "test_acc": test_acc})
    mlflow.log_param("test_samples", len(test_imgs))
    mlflow.log_artifact(str(metrics_path), artifact_path="metrics")
    EVAL_RUN_ID = eval_run.info.run_id
print("Métriques loggées dans MLflow.")


Best model: simple_model
Checkpoint (.keras) : /tmp/tmph0rrdgmi/model/simple_test_model.keras
Modèle chargé pour les tests.
Métriques test sauvegardées.
🏃 View run simple_model-evaluation at: https://dagshub.com/npai.consulting/p08.mlflow/#/experiments/0/runs/290b1f2fb96c4eccad08ddd993e15ac4
🧪 View experiment at: https://dagshub.com/npai.consulting/p08.mlflow/#/experiments/0
Métriques loggées dans MLflow.


## 10. Prédiction & visualisation

In [ ]:
import matplotlib.pyplot as plt

palette = np.load(str(OUT_DIR / "palette_8x3.npy"))

def colorize_mask(mask_np):
    colored = palette[mask_np]
    return colored.astype(np.uint8)

def predict_image(img_path, model, save_dir=OUT_DIR/"predictions", plot=True):
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    img_raw = tf.io.read_file(img_path)
    img = tf.image.decode_png(img_raw, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img_resized = tf.image.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    pred = model.predict(tf.expand_dims(img_resized, 0), verbose=0)[0]
    pred_lbl = tf.argmax(pred, axis=-1).numpy().astype(np.uint8)
    color = colorize_mask(pred_lbl)

    stem = Path(img_path).stem.replace("_leftImg8bit", "")
    out_mask = save_dir / f"{stem}_pred_mask.png"
    out_visu = save_dir / f"{stem}_visu.png"

    tf.keras.utils.save_img(str(out_mask), np.expand_dims(pred_lbl, -1), scale=False)
    tf.keras.utils.save_img(str(out_visu), color, scale=False)

    if plot:
        plt.figure(figsize=(16,6))
        plt.subplot(1,3,1); plt.title("Image"); plt.imshow(img_resized.numpy()); plt.axis("off")
        plt.subplot(1,3,2); plt.title("Mask (labels)"); plt.imshow(pred_lbl, vmin=0, vmax=N_CLASSES-1); plt.axis("off")
        plt.subplot(1,3,3); plt.title("Mask colorisé"); plt.imshow(color); plt.axis("off")
        plt.show()
    return out_mask, out_visu

def predict_test_dataset(model, image_paths, save_dir=OUT_DIR/"predictions", max_visualizations=3):
    generated = []
    for idx, path in enumerate(image_paths):
        plot = idx < max_visualizations
        out_mask, out_visu = predict_image(path, model, save_dir=save_dir, plot=plot)
        generated.append((str(out_mask), str(out_visu)))
    return generated

if len(test_imgs) > 0:
    predictions = predict_test_dataset(best_model, test_imgs, max_visualizations=3)
    predictions_dir = OUT_DIR / "predictions"
    inference_tags = {"stage": "inference", "best_model": best_name}
    parent_for_inference = MLFLOW_TRAINING_RUN_IDS.get(best_name) or globals().get("EVAL_RUN_ID")
    if parent_for_inference:
        inference_tags["parent_run_id"] = parent_for_inference
    with mlflow.start_run(run_name=f"{best_name}-predictions", tags=inference_tags):
        mlflow.log_param("num_predictions", len(predictions))
        if predictions_dir.exists():
            mlflow.log_artifacts(str(predictions_dir), artifact_path="predictions")
    print(f"{len(predictions)} prédictions sauvegardées dans {predictions_dir}.")
else:
    print("Pas d'images test indexées.")


## 11. Exports pour l'API

In [ ]:

print("Contenu de", OUT_DIR)
for p in sorted(OUT_DIR.iterdir()):
    print(" -", p.name)
